<a href="https://colab.research.google.com/github/shinome551/colab/blob/master/temporal_ensembling.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!git clone https://github.com/shinome551/machinelearning.git
!git clone https://github.com/chenyaofo/CIFAR-pretrained-models

Cloning into 'machinelearning'...
remote: Enumerating objects: 9, done.
remote: Counting objects: 100% (9/9), done.
remote: Compressing objects: 100% (7/7), done.
remote: Total 9 (delta 1), reused 0 (delta 0), pack-reused 0
Unpacking objects: 100% (9/9), done.
Cloning into 'CIFAR-pretrained-models'...
remote: Enumerating objects: 41, done.
remote: Total 41 (delta 0), reused 0 (delta 0), pack-reused 41
Unpacking objects: 100% (41/41), done.


In [2]:
import sys
sys.path.append('CIFAR-pretrained-models/cifar_pretrainedmodels/')
sys.path.append('machinelearning/')
import resnet
from utils import *

In [3]:
import os
import datetime
filepath = datetime.datetime.now().strftime('%Y_%m%d_%H%M%S') + '/'
os.mkdir(filepath)

In [62]:
import torch
import torchvision
import torchvision.transforms as transforms
from torchvision.datasets import CIFAR10
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import numpy as np
import matplotlib.pyplot as plt
import time
import datetime
import copy
from torch.utils.data import DataLoader
from torch.utils.data.dataset import Subset
device = torch.device('cuda:0')

In [123]:
train_transform = transforms.Compose([
    transforms.RandomCrop(size=32, padding=4),
    transforms.RandomHorizontalFlip(),
    transforms.ToTensor(),
    transforms.Normalize(
        mean=[0.4914, 0.4822, 0.4465], # mean=[0.5071, 0.4865, 0.4409] for cifar100
        std=[0.2023, 0.1994, 0.2010], # std=[0.2009, 0.1984, 0.2023] for cifar100
    ),
])

test_transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize(
        mean=[0.4914, 0.4822, 0.4465], # mean=[0.5071, 0.4865, 0.4409] for cifar100
        std=[0.2023, 0.1994, 0.2010], # std=[0.2009, 0.1984, 0.2023] for cifar100
    ),
])

trainset = CIFAR10(root='./data', train=True, download=True, transform=train_transform)
testset = CIFAR10(root='./data', train=False, download=True, transform=test_transform)
train_tag = torch.LongTensor(trainset.targets)

Files already downloaded and verified
Files already downloaded and verified


In [74]:
## データセットのインデックスを取得できるようにデータセットを拡張するクラス
## 叩くとデータ・ラベル・インデックスを返す
class IndexLapper(Dataset):
    def __init__(self, dataset):
        self.dataset = dataset
        
    def __getitem__(self, idx):
        data, target = self.dataset[idx]
        return data, target, idx

    def __len__(self):
        return len(self.dataset)

def train_and_ret_pred(model, optimizer, trainloader, device):
    model.train()
    trainloss = 0
    predicts = torch.tensor([])
    indices_all = torch.LongTensor([]) 
    trainloader_with_idx = DataLoader(IndexLapper(trainloader.dataset), batch_size=trainloader.batch_size, shuffle=True)
    for data in trainloader_with_idx:
        inputs, labels, indices = data
        inputs, labels = inputs.to(device), labels.to(device)
        optimizer.zero_grad()
        outputs = model(inputs)
        loss = F.cross_entropy(outputs, labels)
        loss.backward()
        optimizer.step()
        trainloss += loss.item() * inputs.size()[0]
        ## return predicts and indices
        pred = outputs.data.log_softmax(1).exp()
        predicts = torch.cat([predicts, pred.data.cpu()], 0)
        indices_all = torch.cat([indices_all, indices], 0)
    
    trainloss = trainloss / len(trainloader.dataset)
    return trainloss, predicts, indices_all

In [165]:
def operate(alpha):
    model = resnet.cifar_resnet20(pretrained=None)
    model.to(device)

    optimizer = optim.SGD(model.parameters(), lr=0.1, momentum=0.9, dampening=0, weight_decay=1e-4, nesterov=True)
    lr_scheduler = optim.lr_scheduler.CosineAnnealingLR(optimizer, T_max=200,eta_min=0.001)

    batchsize = 256
    trainloader = DataLoader(trainset, batch_size=batchsize, shuffle=True)
    testloader = DataLoader(testset, batch_size=batchsize, shuffle=False)

    start = time.time()
    num_epochs = 200
    record = defaultdict(lambda: defaultdict(list))
    top_acc = 0
    start = time.time()
    for epoch in range(num_epochs):
        trainloss, predicts, indices = train_and_ret_pred(model, optimizer, trainloader, device)
        current_predicts = predicts[torch.argsort(indices)]
        accuracy = test(model, testloader, device)
        record['key']['trainloss'].append(trainloss)
        record['key']['accuracy'].append(accuracy)
        print('epoch:{}, trainloss:{:.3f}, accuracy:{:.1f}%'.format(epoch + 1, trainloss, accuracy), end='\r')
        lr_scheduler.step()
        if top_acc < accuracy:
            top_acc = accuracy
            best_state = copy.deepcopy(model.state_dict())
        ## EMA and variance
        if epoch == 0:
            EMA_pred = current_predicts
            EMA_pred_squ = current_predicts ** 2
            EMA_var = EMA_pred_squ - EMA_pred ** 2
        else:
            EMA_pred = alpha * EMA_pred + (1-alpha) * current_predicts
            EMA_pred_squ = alpha * EMA_pred_squ + (1-alpha) * current_predicts ** 2
            EMA_var = EMA_pred_squ - EMA_pred ** 2

    print('')
    print('epoch per time:{:.3f}s, top acc:{:.1f}%'.format((time.time() - start) / num_epochs, top_acc))
    record['key']['EMA_pred'] = EMA_pred
    record['key']['EMA_var'] = EMA_var
    return model, best_state, record

In [166]:
alpha = 0.9
model, best_state, record = operate(alpha)

KeyboardInterrupt: ignored